In [24]:
import pandas as pd
import json
import sqlite3

In [39]:
# Load CSV
orders = pd.read_csv("orders (4).csv")

In [40]:
# Convert order_date to datetime for better analysis
orders_df['order_date'] = pd.to_datetime(orders_df['order_date'], dayfirst=True)

print("Orders Loaded. Shape:", orders_df.shape)
orders_df.head()

Orders Loaded. Shape: (10000, 6)


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,2023-02-18,842.97,New Foods Chinese
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,2023-07-15,163.93,Spice Kitchen Punjabi
3,4,319,224,2023-10-04,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,2023-12-25,1321.91,Royal Eatery South Indian


In [41]:
import json

# Load the users dataset
with open('users (1).json', 'r') as f:
    users_data = json.load(f)

users_df = pd.DataFrame(users_data)

print("Users Loaded. Shape:", users_df.shape)
users_df.head()

Users Loaded. Shape: (3000, 4)


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [42]:
import re

# Read the SQL file content
with open('restaurants (1).sql', 'r') as f:
    sql_content = f.read()

# Extract values using regex pattern: (id, 'name', 'cuisine', rating)
pattern = r"\((\d+),\s*'([^']*)',\s*'([^']*)',\s*([\d\.]+)\)"
matches = re.findall(pattern, sql_content)

# Create DataFrame
restaurants_df = pd.DataFrame(matches, columns=['restaurant_id', 'restaurant_name_sql', 'cuisine', 'rating'])

# Convert data types
restaurants_df['restaurant_id'] = restaurants_df['restaurant_id'].astype(int)
restaurants_df['rating'] = restaurants_df['rating'].astype(float)

print("Restaurants Loaded. Shape:", restaurants_df.shape)
restaurants_df.head()

Restaurants Loaded. Shape: (500, 4)


,restaurant_id,restaurant_name_sql,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [43]:
# Join 1: Merge Orders with Users (on user_id)
merged_df = orders_df.merge(users_df, on='user_id', how='left')

# Join 2: Merge the result with Restaurants (on restaurant_id)
final_df = merged_df.merge(restaurants_df, on='restaurant_id', how='left')

# Clean up: You might have duplicate restaurant name columns
# We'll keep the one from the orders file as it's more descriptive
final_df.drop(columns=['restaurant_name_sql'], inplace=True)

print("Merging Complete. Final Shape:", final_df.shape)
final_df.head()

Merging Complete. Final Shape: (10000, 11)


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership,cuisine,rating
0,1,2508,450,2023-02-18,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Mexican,3.2
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Indian,4.5
2,3,2084,107,2023-07-15,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Mexican,4.0
3,4,319,224,2023-10-04,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Chinese,4.8
4,5,1064,293,2023-12-25,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Italian,3.0


In [44]:
#Cretae final Dataset

In [45]:
final_df.to_csv('final_food_delivery_dataset.csv', index=False)
print("File 'final_food_delivery_dataset.csv' has been created successfully!")

File 'final_food_delivery_dataset.csv' has been created successfully!


In [46]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   order_id         10000 non-null  int64         
 1   user_id          10000 non-null  int64         
 2   restaurant_id    10000 non-null  int64         
 3   order_date       10000 non-null  datetime64[ns]
 4   total_amount     10000 non-null  float64       
 5   restaurant_name  10000 non-null  object        
 6   name             10000 non-null  object        
 7   city             10000 non-null  object        
 8   membership       10000 non-null  object        
 9   cuisine          10000 non-null  object        
 10  rating           10000 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(5)
memory usage: 859.5+ KB


In [47]:
# Filter for Gold members
gold_members = final_df[final_df['membership'] == 'Gold']

# Group by city and sum the total_amount
city_revenue = gold_members.groupby('city')['total_amount'].sum().sort_values(ascending=False)

print(city_revenue)

city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64


In [49]:
# Average order value by cuisine
result = final_df.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)

print(result)

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64


In [50]:
# Count of distinct users whose total order sum exceeds 1000
print((final_df.groupby('user_id')['total_amount'].sum() > 1000).sum())

2544


In [57]:
# Create rating ranges (bins) from 0 to 5
bins = [0, 1, 2, 3, 4, 5]
final_df['rating_range'] = pd.cut(final_df['rating'], bins=bins)

# Group by the range and sum the revenue to find the highest
result = final_df.groupby('rating_range')['total_amount'].sum().sort_values(ascending=False)

print(result)

rating_range
(4, 5]    4157357.01
(3, 4]    3599248.98
(2, 3]     255018.13
(0, 1]          0.00
(1, 2]          0.00
Name: total_amount, dtype: float64


C:\Users\VISHAL\AppData\Local\Temp\ipykernel_3820\1209918957.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  result = final_df.groupby('rating_range')['total_amount'].sum().sort_values(ascending=False)


In [59]:
# Calculate average order value for Gold members by city
result = final_df[final_df['membership'] == 'Gold'].groupby('city')['total_amount'].mean().sort_values(ascending=False)

print(result)

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64


In [60]:
# Group by cuisine to see unique restaurant count and total revenue
cuisine_summary = final_df.groupby('cuisine').agg(
    restaurant_count=('restaurant_id', 'nunique'),
    total_revenue=('total_amount', 'sum')
).sort_values(by='restaurant_count')

print(cuisine_summary)

         restaurant_count  total_revenue
cuisine                                 
Chinese               120     1930504.65
Indian                126     1971412.58
Italian               126     2024203.80
Mexican               128     2085503.09


In [61]:
# Percentage of orders by Gold members rounded to nearest integer
print(round(final_df['membership'].value_counts(normalize=True)['Gold'] * 100))

50


In [70]:
# 1. Merge correctly with clear suffixes
# This creates 'restaurant_name_sql' for the names like 'Restaurant_294'
final_df = orders_df.merge(users_df, on='user_id', how='left')
final_df = final_df.merge(restaurants_df, on='restaurant_id', how='left', suffixes=('_csv', '_sql'))

# 2. Group by the SQL restaurant name and calculate count and mean
res = final_df.groupby('restaurant_name_sql')['total_amount'].agg(['count', 'mean'])

# 3. Filter for less than 20 orders and sort by average value (mean)
res = res[res['count'] < 20].sort_values('mean', ascending=False)

# 4. Rename columns and index to match your request
res.columns = ['total_orders', 'avg_order_value']
res.index.name = 'restaurant_name'

# 5. Display the result
res

,total_orders,avg_order_value
restaurant_name,,
Restaurant_294,13,1040.222308
Restaurant_262,18,1029.473333
Restaurant_77,12,1029.180833
Restaurant_193,15,1026.306667
Restaurant_7,16,1002.140625
...,...,...
Restaurant_184,19,621.828947
Restaurant_498,18,596.815556
Restaurant_192,14,589.972857


In [72]:
# Group by membership and cuisine, sum total_amount, and sort descending
result = final_df.groupby(['membership', 'cuisine'])['total_amount'].sum().sort_values(ascending=False)

print(result)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64


In [73]:
# Convert order_date to datetime format
final_df['order_date'] = pd.to_datetime(final_df['order_date'], dayfirst=True)

# Extract the quarter (1, 2, 3, or 4)
final_df['quarter'] = final_df['order_date'].dt.quarter

# Group by quarter and sum the revenue
quarterly_revenue = final_df.groupby('quarter')['total_amount'].sum().sort_values(ascending=False)

print(quarterly_revenue)

quarter
3    2037385.10
4    2018263.66
1    2010626.64
2    1945348.72
Name: total_amount, dtype: float64


In [83]:
# Count total orders placed by Gold members
print((final_df['membership'] == 'Gold').sum())

4987


1889367


In [78]:
print(final_df['user_id'].nunique())

2883


In [79]:
avg_gold_order = final_df[final_df['membership'] == 'Gold']['total_amount'].mean()

print(round(avg_gold_order, 2))

797.15


In [80]:
final_df['rating'] = final_df['rating'].astype(float)
high_rating_orders = (final_df['rating'] >= 4.5).sum()

print(high_rating_orders)

3374


In [82]:
gold_data = final_df[final_df['membership'] == 'Gold']
top_city = gold_data.groupby('city')['total_amount'].sum().idxmax()
order_count = gold_data[gold_data['city'] == top_city].shape[0]

print(f"Top City: {top_city}")
print(f"Total Orders: {order_count}")

Top City: Chennai
Total Orders: 1337
